In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import os

In [ ]:
def load_file(file_path):
    if file_path[-3:] == 'tsv':
        df = pd.read_csv(file_path, sep="\t")
        df = df[df.iloc[:,0].apply(lambda x: x[-2:]) == 'SK'].transpose()
        df.columns = df.iloc[0] 
        return df[1:]
    if file_path[-3:] == 'csv':
        return pd.read_csv(file_path)

In [ ]:
datasets = [load_file('data/' + file) for file in os.listdir("data/")]